### Finding out where `qiskit.{...}.Isometry` gives us a quantum circuit to implement the isometry 

In [21]:
import numpy as np

from qiskit import QuantumCircuit, transpile
from qiskit.extensions.quantum_initializer import Isometry
from qiskit import quantum_info as qi
from qiskit_aer import AerSimulator

In [17]:
backend = AerSimulator()

In [10]:
A = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])
A = A.reshape((4, 2))
A

array([[1, 0],
       [0, 0],
       [0, 1],
       [0, 0]])

In [13]:
# confirm that A is a left isometry
print(np.matmul(A.T, A))
print(np.matmul(A, A.T))

[[1 0]
 [0 1]]
[[1 0 0 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 0]]


In [18]:
qc = QuantumCircuit(2)
qc.append(Isometry(A, 0, 0), [0, 1])
qc_compiled = transpile(qc, backend)

In [20]:
qc_compiled.draw()

global phase: π/2
     ┌──────────────────────────────┐     ┌─────────┐┌───┐┌─────────┐
q_0: ┤ Diagonal(0.70711-0.70711j,1) ├──■──┤ Unitary ├┤ X ├┤ Unitary ├
     └─────────┬─────────┬──────────┘┌─┴─┐├─────────┤└─┬─┘└─────────┘
q_1: ──────────┤ Unitary ├───────────┤ X ├┤ Unitary ├──■─────────────
               └─────────┘           └───┘└─────────┘

In [23]:
op = qi.Operator(qc_compiled)
U = op.data

In [24]:
U, U.shape

(array([[ 1.00000000e+00-7.14959896e-16j,  1.96261557e-17-1.96261557e-17j,
         -1.96261557e-16+1.96261557e-16j, -9.00013588e-17+1.66533454e-16j],
        [ 0.00000000e+00-1.04576541e-16j, -2.15887713e-16+2.15887713e-16j,
         -3.92523115e-17+0.00000000e+00j, -7.88413545e-16-1.00000000e+00j],
        [ 3.92523115e-17+1.96261557e-17j,  1.00000000e+00-7.98371058e-16j,
         -1.11022302e-16+4.09359695e-17j, -7.85046229e-17+9.81307787e-17j],
        [ 1.17756934e-16+9.81307787e-17j,  1.66533454e-16-8.00438459e-17j,
         -1.00000000e+00+8.47292013e-16j,  1.23259516e-32+1.96261557e-17j]]),
 (4, 4))

In [36]:
print(np.round(np.matmul(U.T, U), 2))
print(np.round(np.matmul(U, U.T), 2))

[[ 1.-0.j  0.+0.j -0.+0.j -0.+0.j]
 [ 0.+0.j  1.-0.j -0.+0.j  0.+0.j]
 [-0.+0.j -0.+0.j  1.-0.j -0.+0.j]
 [-0.+0.j  0.+0.j -0.+0.j -1.+0.j]]
[[ 1.-0.j  0.-0.j  0.-0.j  0.-0.j]
 [ 0.-0.j -1.+0.j -0.+0.j  0.-0.j]
 [ 0.-0.j -0.+0.j  1.-0.j  0.-0.j]
 [ 0.-0.j  0.-0.j  0.-0.j  1.-0.j]]


### The above is very close to identity but has a factor of -1 in some places 

In [30]:
print(np.allclose(np.eye(4), np.round(np.matmul(U, U.T)), atol=1e-3))
print(np.allclose(np.eye(4), np.round(np.matmul(U.T, U)), atol=1e-3))

False
False
